# This Notebook converts data to lcdata format

basically the way you should provide the data in input to generate the prediction is a table in lcdata  format, which is explained here https://lcdata.readthedocs.io/en/latest/usage.html (edited)
[2:16 PM] There is a metadata table in it which needs the following keys
- object_id: A unique identifer. Default: randomly assigned string
- ra: The right ascension. Default: nan
- dec: The declination. Default: nan
- type: A string representing the type of the light curve. Default: Unknown
- redshift: The redshift. Default: nan
(edited)

[2:16 PM] And a lightcurve table with
- time: times at which the light curve was sampled. Converted to a 64-bit float.
- flux: The flux at each point on the light curve. Converted to a 32-bit float.
- fluxerr: The uncertainty on the flux. Converted to a 32-bit float.
- band: A string representing bandpass that the light curve was observed in. We recommend using the sncosmo bandpass names here. Converted to a binary string.

In [150]:
import parsnip
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json
import lcdata
import os
import pickle
from astropy.io import fits
from astropy.table import Table, Column, join, hstack, vstack, unique, setdiff
import h5py

path = '/Users/EmilieCote/Research/ULAB_Mentor_23/NewData/newest_data_DCDE2/newest_data/'


In [151]:
directory = os.fsencode(path)
transient_files = []
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv") and os.path.exists(path+filename[:-4]+'.pkl'): 
        transient_files.append(filename[:-4])

In [152]:
transient_files

['T202205152126433p055058',
 'T202205121511181p055933',
 'A202205152124144p054322',
 'A202205031501334p062801',
 'A202205152129520p052135',
 'A202206082110219p071900',
 'A202205152134324p041154',
 'T202204241256226p063126',
 'T202204211426361m013101',
 'A202204241256100p051750',
 'A202205152130004p064115',
 'A202204211246163p072445',
 'A202206082109023p041842',
 'A202204211458526p073919',
 'A202204211450104p084554',
 'A202204241515454p073647',
 'A202204211426390m023211',
 'A202204211259539p060050',
 'T202204211253407p055203',
 'A202204241421240m003354',
 'T202204211244049p054952',
 'A202204211306143p071035',
 'T202204211301485p061149',
 'A202205152133539p050340',
 'A202204211309243p070809',
 'T202205181256397p052742',
 'A202204211306293p055323',
 'A202205272107014p041648',
 'A202204241505191p070347',
 'A202204211305486p071217',
 'A202205272124005p064132',
 'A202204211416240m005231',
 'A202204241507298p064415',
 'A202204211426017m020733',
 'A202204211507100p060457',
 'A202204211450585p0

In [153]:
len(transient_files)

535

In [154]:
class transient():
    light_curve = None
    
    def __init__(self, data, meta):
        self.data = data
        self.meta = meta

In [155]:
transient_list = []
for file in transient_files:
    with open(path + file +'.pkl', 'rb') as fp:
        tran_dict = pickle.load(fp)
    data = pd.read_csv(path + file +'.csv')
    tr = transient(data, tran_dict)
    transient_list.append(tr)

In [156]:
transient_list[4].data.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'col0', 'XWIN_IMAGE', 'YWIN_IMAGE',
       'SNR_WIN', 'MAGERR_APER', 'XWIN_WORLD', 'YWIN_WORLD', 'DetOnWhich',
       'legacyGalRa', 'legacyGalDec', 'legacyGalType', 'legacyGalgMag',
       'legacyGalrMag', 'legacyGalzMag', 'legacyGalgMagErr',
       'legacyGalrMagErr', 'legacyGalzMagErr', 'legacyStaRa', 'legacyStaDec',
       'legacyStagMag', 'legacyStagMagErr', 'legacyStarMag',
       'legacyStarMagErr', 'legacyStazMag', 'legacyStazMagErr', 'EigVal1',
       'EigVal2', 'REAL_SCORE', 'MAG_APERc', 'SEP2STAR', 'SEP2GAL',
       'MPNAME_oREF', 'MPVMAG_oREF', 'MPSEP_oREF', 'MPNAME_SCI', 'MPVMAG_SCI',
       'MPSEP_SCI', 'DIFNAME', 'SCINAME', 'OREFNAME', 'REFNAME', 'MJD_OBS_SCI',
       'MJD_OBS_oREF', 'DETPOS', 'LM5_oREF', 'LM5_SCI', 'ZP_DIFF', 'EZP_DIFF',
       'OAPER_DIFF', 'AlertID', 'ROUND', 'ObjectID'],
      dtype='object')

In [157]:
transient_list[2].data[['MAG_APERc','MAGERR_APER','ZP_DIFF', 'EZP_DIFF','legacyGalgMag',
       'legacyGalrMag', 'legacyGalzMag', 'legacyGalgMagErr',
    'legacyGalrMagErr', 'legacyGalzMagErr',
       'legacyStagMag', 'legacyStagMagErr', 'legacyStarMag',
       'legacyStarMagErr', 'legacyStazMag', 'legacyStazMagErr']]

,MAG_APERc,MAGERR_APER,ZP_DIFF,EZP_DIFF,legacyGalgMag,legacyGalrMag,legacyGalzMag,legacyGalgMagErr,legacyGalrMagErr,legacyGalzMagErr,legacyStagMag,legacyStagMagErr,legacyStarMag,legacyStarMagErr,legacyStazMag,legacyStazMagErr
0,21.222935,0.121293,28.858993,0.008460,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
1,21.196419,0.038887,29.414225,0.007667,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
2,21.398071,0.122331,29.599081,0.007662,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
3,21.183086,0.043638,29.531099,0.011811,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
4,21.372173,0.042190,29.412441,0.009088,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
5,21.438660,0.078664,29.567581,0.009887,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
6,21.350370,0.106367,28.908742,0.009561,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
7,21.309015,0.095812,28.836649,0.008456,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
8,21.280653,0.087391,29.413781,0.008413,21.091528,20.819490,19.999897,0.011061,0.013266,0.013711,27.124844,1.711704,25.377350,0.600100,22.626118,0.113302
9,20.707218,0.079646,29.642093,0.007419,24.617512,24.290388,22.986769,0.192897,0.217893,0.146694,21.126884,0.008794,19.637232,0.003843,17.905399,0.001634


In [158]:
transient_list[2].data[['MAGERR_APER','REAL_SCORE',
       'MAG_APERc', 'SEP2STAR', 'SEP2GAL', 'MPNAME_oREF', 'MPVMAG_oREF',
       'MPSEP_oREF', 'MPNAME_SCI', 'MPVMAG_SCI', 'MPSEP_SCI', 'EZP_DIFF',
       'LM5_SCI', 'LM5_oREF', 'OAPER_DIFF', 'ZP_DIFF', 'DIFNAME',
       'MJD_OBS_SCI', 'MJD_OBS_oREF', 'OREFNAME', 'REFNAME', 'SCINAME',
       'DETPOS', 'col0', 'AlertID', 'ROUND']]

,MAGERR_APER,REAL_SCORE,MAG_APERc,SEP2STAR,SEP2GAL,MPNAME_oREF,MPVMAG_oREF,MPSEP_oREF,MPNAME_SCI,MPVMAG_SCI,...,DIFNAME,MJD_OBS_SCI,MJD_OBS_oREF,OREFNAME,REFNAME,SCINAME,DETPOS,col0,AlertID,ROUND
0,0.121293,0.964,21.222935,7.977208,0.107329,NONE,NaN,NaN,NONE,NaN,...,gst_220515_081325_ooi_g_desirt.fits.fz,59714.342661,56888.099033,c4d_140819_022500_ooi_g_ls9.fits.fz,rtp_220515_081325_ooi_g_desirt.fits.fz,c4d_220515_081325_ooi_g_desirt.fits.fz,S29,7.0,220515081325S292124144961p0543229537,8.0
1,0.038887,0.918,21.196419,7.957200,0.088698,NONE,NaN,NaN,NONE,NaN,...,gst_220515_081444_ooi_r_desirt.fits.fz,59714.343568,56886.189854,c4d_140817_043530_ooi_r_ls9.fits.fz,rtp_220515_081444_ooi_r_desirt.fits.fz,c4d_220515_081444_ooi_r_desirt.fits.fz,S29,20.0,220515081444S292124144940p0543228843,8.0
2,0.122331,0.985,21.398071,8.036858,0.285402,NONE,NaN,NaN,NONE,NaN,...,gst_220515_081622_ooi_z_desirt.fits.fz,59714.344708,57612.131179,c4d_160812_031111_ooi_z_ls9.fits.fz,rtp_220515_081622_ooi_z_desirt.fits.fz,c4d_220515_081622_ooi_z_desirt.fits.fz,S29,18.0,220515081622S292124145020p0543231434,8.0
3,0.043638,0.951,21.183086,7.923423,0.076962,NONE,NaN,NaN,NONE,NaN,...,gst_220515_083435_ooi_g_desirt.fits.fz,59714.357353,56885.116493,c4d_140816_025016_ooi_g_ls9.fits.fz,rtp_220515_083435_ooi_g_desirt.fits.fz,c4d_220515_083435_ooi_g_desirt.fits.fz,N28,3472.0,220515083435N282124144946p0543228438,8.0
4,0.042190,0.944,21.372173,7.900994,0.052910,NONE,NaN,NaN,NONE,NaN,...,gst_220515_083553_ooi_r_desirt.fits.fz,59714.358256,56885.128405,c4d_140816_030703_ooi_r_ls9.fits.fz,rtp_220515_083553_ooi_r_desirt.fits.fz,c4d_220515_083553_ooi_r_desirt.fits.fz,N28,4241.0,220515083553N282124144964p0543228402,8.0
5,0.078664,0.988,21.438660,7.846649,0.161748,NONE,NaN,NaN,NONE,NaN,...,gst_220515_083732_ooi_z_desirt.fits.fz,59714.359402,56885.175965,c4d_140816_041620_ooi_z_ls9.fits.fz,rtp_220515_083732_ooi_z_desirt.fits.fz,c4d_220515_083732_ooi_z_desirt.fits.fz,N28,NaN,NaN,NaN
6,0.106367,0.944,21.350370,8.000640,0.135884,NONE,NaN,NaN,NONE,NaN,...,gst_220515_085421_ooi_g_desirt.fits.fz,59714.371086,56888.099033,c4d_140819_022500_ooi_g_ls9.fits.fz,rtp_220515_085421_ooi_g_desirt.fits.fz,c4d_220515_085421_ooi_g_desirt.fits.fz,S29,10.0,220515085421S292124144958p0543229828,8.0
7,0.095812,0.961,21.309015,7.813213,0.141890,NONE,NaN,NaN,NONE,NaN,...,gst_220515_085940_ooi_g_desirt.fits.fz,59714.374769,56885.116493,c4d_140816_025016_ooi_g_ls9.fits.fz,rtp_220515_085940_ooi_g_desirt.fits.fz,c4d_220515_085940_ooi_g_desirt.fits.fz,N28,1655.0,220515085940N282124145084p0543229149,8.0
8,0.087391,0.972,21.280653,7.927542,0.191437,NONE,NaN,NaN,NONE,NaN,...,gst_220518_082416_ooi_r_desirt.fits.fz,59717.350196,56886.189854,c4d_140817_043530_ooi_r_ls9.fits.fz,rtp_220518_082416_ooi_r_desirt.fits.fz,c4d_220518_082416_ooi_r_desirt.fits.fz,S29,19.0,220518082416S292124145052p0543230328,9.0
9,0.079646,0.244,20.707218,0.733492,3.898770,NONE,NaN,NaN,NONE,NaN,...,gst_220518_082556_ooi_z_desirt.fits.fz,59717.351348,57612.131179,c4d_160812_031111_ooi_z_ls9.fits.fz,rtp_220518_082556_ooi_z_desirt.fits.fz,c4d_220518_082556_ooi_z_desirt.fits.fz,S29,NaN,NaN,NaN


In [159]:
transient_list[60].meta

{'ObjectID': 'A202204211305051p063549',
 'RA-OBJECT': 196.27143643,
 'DEC-OBJECT': 6.59711708,
 'NumberAlerts': 19,
 'MaxSCORE': 0.83,
 'RA-PSEUDO-HOST': 196.27137041,
 'DEC-PSEUDO-HOST': 6.59714711,
 'SEP-PSEUDO-HOST': 0.2597,
 'RA-NEIGHBOR-STAR': 196.27368193,
 'DEC-NEIGHBOR-STAR': 6.59647697,
 'SEP-NEIGHBOR-STAR': 8.3544,
 'Discovery-Round': 0,
 'Discovery-Time': '2022-04-21T04:32:17.467',
 'Discovery-Filter': 'S',
 'Discovery-Magnitude': 22.06,
 'Discovery-SNR': 11.3,
 'Latest-Round': 13,
 'Latest-Time': '2022-05-30T01:12:08.238',
 'Latest-Filter': 'S',
 'Latest-Magnitude': 22.3,
 'Latest-SNR': 8.6,
 'z_spec': -99.0,
 'z_photo_mean': 0.8296526,
 'z_photo_err': 0.20829105}

In [161]:
transient_list[3].data['SCINAME'][3][22]

'z'

In [8]:
# Old
def convert_data(data):
    data_copy = data[['MJD_OBS_SCI', 'legacyStagMag', 'legacyStagMagErr', 'legacyStaRa', 'legacyStaDec', 'legacyGalRa', 'legacyGalDec']]
    data_copy['legacyStagMag'] = data_copy['legacyStagMag'].apply(lambda x: (10**22.5-x)/2.5)  #converting from magnitude to flux
    data_copy['legacyStagMagErr'] = data_copy['legacyStagMag'] * np.log(10) / 2.5 * data['legacyStagMagErr'] #'legacyStagMag' here is the flux
    data_copy = data_copy.rename(columns={'legacyStagMag':'StagFlux', 'legacyStagMagErr':'StagFluxErr'})
    data_copy['band'] = range(len(data_copy))
    for i in range(len(data_copy)):
        bandfilter = list(data['SCINAME'])[i][-16] #this is how we get the band (filter) to add to our list
        data_copy.loc[i,'band'] = bandfilter
    return data_copy

In [32]:
def _convert_data(data):
    data_copy = data[['MJD_OBS_SCI', 'legacyStagMag', 'legacyStagMagErr', 'legacyStaRa', 'legacyStaDec', 'legacyGalRa', 'legacyGalDec']]
    data_copy = data_copy.rename(columns={'legacyStagMag':'StaFlux', 'legacyStagMagErr':'StaFluxErr'})
    data_copy['band'] = range(len(data_copy))
    for i in range(len(data_copy)):
        bandfilter = list(data['SCINAME'])[i][-16] #this is how we get the band (filter) to add to our list
        data_copy.loc[i,'band'] = bandfilter
        data_copy.loc[i,'StaFlux'] = (-data['legacySta'+bandfilter+'Mag'][i] + 10**22.5) / 2.5
        data_copy.loc[i,'StaFluxErr'] = data['legacySta'+bandfilter+'Mag'][i] * np.log(10) / 2.5 * data['legacySta'+bandfilter+'MagErr'][i]
    return data_copy

# Fixed?

def convert_data(data):
    data_copy = data[['MJD_OBS_SCI', 'legacyStagMag', 'legacyStagMagErr', 'legacyStaRa', 'legacyStaDec', 'legacyGalRa', 'legacyGalDec']]
    data_copy = data_copy.rename(columns={'legacyStagMag':'StaFlux', 'legacyStagMagErr':'StaFluxErr'})
    data_copy['band'] = range(len(data_copy))
    for i in range(len(data_copy)):
        bandfilter = list(data['SCINAME'])[i][22] #this is how we get the band (filter) to add to our list
        if bandfilter != 'g' and bandfilter != 'z' and bandfilter != 'r':
            print(data['SCINAME'][i])
        data_copy.loc[i,'band'] = bandfilter
        data_copy.loc[i,'StaFlux'] = 10**((22.5-data['legacySta'+bandfilter+'Mag'][i])/2.5)
        data_copy.loc[i,'StaFluxErr'] = data['legacySta'+bandfilter+'Mag'][i] * np.log(10) / 2.5 * data['legacySta'+bandfilter+'MagErr'][i]
    return data_copy

def convert_data_3timesRow(data):
    data_copy2 = data[['MJD_OBS_SCI', 'legacyStagMag', 'legacyStagMagErr', 'legacyStaRa', 'legacyStaDec', 'legacyGalRa', 'legacyGalDec']]
    data_copy = pd.concat([data_copy2,data_copy2,data_copy2])
    data_copy = data_copy.reset_index()
    data_copy = data_copy.rename(columns={'legacyStagMag':'StaFlux', 'legacyStagMagErr':'StaFluxErr'})
    data_copy['band'] = range(len(data_copy))
    for i in range(len(data_copy2)):
        data_copy.loc[i,'StaFlux'] = 10**((22.5-data['legacyStagMag'][i])/2.5)
        data_copy.loc[i + len(data),'StaFlux'] = 10**((22.5-data['legacyStarMag'][i])/2.5)
        data_copy.loc[i + 2*len(data),'StaFlux'] = 10**((22.5-data['legacyStazMag'][i])/2.5)
        data_copy.loc[i,'StaFluxErr'] = data['legacyStagMag'][i] * np.log(10) / 2.5 * data['legacyStagMagErr'][i]
        data_copy.loc[i + len(data),'StaFluxErr'] = data['legacyStarMag'][i] * np.log(10) / 2.5 * data['legacyStarMagErr'][i]
        data_copy.loc[i + 2*len(data),'StaFluxErr'] = data['legacyStazMag'][i] * np.log(10) / 2.5 * data['legacyStazMagErr'][i]
        data_copy.loc[i, 'band'] = 'g'
        data_copy.loc[i + len(data), 'band'] = 'r'
        data_copy.loc[i + 2*len(data), 'band'] = 'z'
    return data_copy

In [162]:
# Using 'MAG_APERc','MAGERR_APER'

def convert_data_Aper(data):
    data_copy = data[['MAG_APERc','MAGERR_APER','MJD_OBS_SCI', 'legacyStagMag', 'legacyStagMagErr', 'legacyStaRa', 'legacyStaDec', 'legacyGalRa', 'legacyGalDec']]
    data_copy = data_copy.rename(columns={'MAG_APERc':'Flux', 'MAGERR_APER':'FluxErr'})
    data_copy['band'] = range(len(data_copy))
    for i in range(len(data_copy)):
        bandfilter = list(data['SCINAME'])[i][22] #this is how we get the band (filter) to add to our list
        if bandfilter != 'g' and bandfilter != 'z' and bandfilter != 'r':
            print(data['SCINAME'][i])
        data_copy.loc[i,'band'] = bandfilter
        data_copy.loc[i,'Flux'] = 10**((22.5-data['MAG_APERc'][i])/2.5)
        data_copy.loc[i,'FluxErr'] = data_copy['Flux'][i] * np.log(10) / 2.5 * data['MAGERR_APER'][i]
    return data_copy

$$F = 10^{-\frac{M}{2.5}}$$
$$Flux = 10^{-\frac{Magnitude}{ZP}}$$
Where ZP = Zero Point

## $$\textbf{Magnitude} = 22.5− 2.5log_{10}(\textbf{Flux})$$

 

$$2.5 log_{10}(F)=22.5-M $$
$$log_{10}(F)=(22.5-M)/2.5$$

$$$$

## $$\textbf{Flux}=10^{\frac{22.5-\textbf{Magnitude}}{2.5}} $$


## $$\text{Flux}=10^{\frac{22.5-\textbf{Magnitude}}{2.5}} $$, where 22.5 is our Z.P.

In [43]:
#First Function

def _create_lc_data(data, meta):
    if meta['z_spec'] == -99:
        flux = list(10**((22.5-data['legacyStagMag'])/2.5))
        magerr = list(data['legacyStagMagErr'])
        fluxerr = np.array(flux) * np.log(10) / 2.5 * np.array(magerr)

        light_curve = Table({
            'time': list(data['MJD_OBS_SCI']),
            'flux': flux,
            'fluxerr': list(fluxerr),
            'band': ['lsst'+i[-16] for i in list(data['SCINAME'])]})
            'zp': list(np.full(len(flux),22.5)),
            'magsys': list(np.full(len(flux),'ab'))})

        light_curve.meta = {
            'object_id': meta['ObjectID'],
            'ra': meta['RA-OBJECT'],
            'dec': meta['DEC-OBJECT'],
            'ra_host': meta['RA-PSEUDO-HOST'],
            'dec_host': meta['DEC-PSEUDO-HOST'],
            'type': 'Unknown',
            'redshift': meta['z_spec'],
            'hostgal_photoz': meta['z_photo_mean'],
            'hostgal_photoz_err': meta['z_photo_err']}
        return light_curve
    else: return None

In [118]:
# includes all transients but keeps -99 as redshift value
    
def create_lc_data2(data, meta, keep_missing_ZSPEC=0):
    
    # includes all transients but keeps -99 as redshift value
    
    if keep_missing_ZSPEC or meta['z_spec'] != -99:
        light_curve = Table({
            'time': list(data['MJD_OBS_SCI']),
            'flux': list(data['Flux']),
            'fluxerr': list(data['FluxErr']),
            'band': list(data['band'])})
            'zp': list(np.full(len(list(data['Flux'])),22.5))
            'magsys': list(np.full(len(flux),'ab'))

        light_curve.meta = {
            'object_id': meta['ObjectID'],
            'ra': meta['RA-OBJECT'],
            'dec': meta['DEC-OBJECT'],
            'ra_host': meta['RA-PSEUDO-HOST'],
            'dec_host': meta['DEC-PSEUDO-HOST'],
            'type': 'Unknown',
            'redshift': meta['z_spec'],
            'hostgal_photoz': meta['z_photo_mean'],
            'hostgal_photoz_err': meta['z_photo_err']}
        return light_curve
    
    else: return None


# Try Convert_data for 1 transient
trans1 = transient_list[0]

new = convert_data_Aper(trans1.data)


In [117]:
len(transient_list)

1086

In [66]:
trans1 = transient_list[2]

new = convert_data_Aper(trans1.data)

In [67]:
sum(new['FluxErr']<new['Flux'])

(45, 45)

In [68]:
new['FluxErr']<new['Flux']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
30    True
31    True
32    True
33    True
34    True
35    True
36    True
37    True
38    True
39    True
40    True
41    True
42    True
43    True
44    True
dtype: bool

In [163]:
# Use photoz if zspec is missing

def create_lc_data3(data, meta):
    # here the redshift is replaced with the photoz value if spectroscopic doesn't exist
    
        light_curve = Table({
            'time': list(data['MJD_OBS_SCI']),
            'flux': list(data['Flux']),
            'fluxerr': list(data['FluxErr']),
            'band': list(data['band'])})
            'zp': list(np.full(len(list(data['StaFlux'])),22.5))
            'magsys': list(np.full(len(flux),'ab'))

        if meta['z_spec'] == -99 and meta['z_photo_mean'] != -99:
            redshift = meta['z_photo_mean']
            z_spec_exists = 0
        elif meta['z_spec'] != -99: 
            redshift = meta['z_spec']
            z_spec_exists = 1
        else: return None
        
        light_curve.meta = {
            'object_id': meta['ObjectID'],
            'ra': meta['RA-OBJECT'],
            'dec': meta['DEC-OBJECT'],
            'ra_host': meta['RA-PSEUDO-HOST'],
            'dec_host': meta['DEC-PSEUDO-HOST'],
            'type': 'Unknown',
            'redshift': redshift,
            'hostgal_photoz': meta['z_photo_mean'],
            'hostgal_photoz_err': meta['z_photo_err'],
            'z_spec exists': z_spec_exists}
        return light_curve



# Try Convert_data for 1 transient 
trans1 = transient_list[4]

new = convert_data_Aper(trans1.data)

In [165]:
new['FluxErr']<new['Flux']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
30    True
31    True
32    True
33    True
34    True
35    True
36    True
37    True
38    True
39    True
40    True
41    True
42    True
43    True
dtype: bool

In [26]:
for lc,tran in enumerate(transient_list):
    tran.light_curve = create_lc_data3(convert_data_3timesRow(tran.data), tran.meta)
    #print(lc)
    
lc_list = [i.light_curve for i in transient_list if i.light_curve]


In [169]:
# create_lc_data2(only Z_SPEC) using Magg_Aperc

for lc,tran in enumerate(transient_list):
    tran.light_curve = create_lc_data2(convert_data_Aper(tran.data), tran.meta, keep_missing_ZSPEC=0)
    print(lc)
    
lc_list_Aper_zspecONLY = [i.light_curve for i in transient_list if i.light_curve]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [170]:
len(lc_list_Aper_zspecONLY)

73

In [166]:
# create_lc_data3(with_PhotoZ) using Magg_Aperc

for lc,tran in enumerate(transient_list):
    tran.light_curve = create_lc_data3(convert_data_Aper(tran.data), tran.meta)
    print(lc)
    
lc_list_Aper = [i.light_curve for i in transient_list if i.light_curve]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [167]:
len(lc_list_Aper)

535

In [168]:
# Write to file create_lc_data3(with_PhotoZ) using Magg_Aperc
# lc_list_Aper_zspecONLY
dataset = lcdata.from_light_curves([i for i in lc_list_Aper])
dataset.write_hdf5('./DCDE2_535_transients_USING_MAG_APERc_photoz_if_NoZSPEC.h5')

In [171]:
# lc_list_Aper_zspecONLY

dataset = lcdata.from_light_curves([i for i in lc_list_Aper_zspecONLY])
dataset.write_hdf5('./DCDE2_73_transients_USING_MAG_APERc_onlyZSPEC.h5')

In [118]:
for lc,tran in enumerate(transient_list):
    tran.light_curve = create_lc_data3(convert_data(tran.data), tran.meta)
    #print(lc)
    
lc_list_OG = [i.light_curve for i in transient_list if i.light_curve]



In [89]:
lc_list_Aper[2]

time,flux,fluxerr,band
float64,float64,float64,str1
59977.304456178135,1.2908202910904998,0.08715988736046605,g
59977.305372998846,1.6432064791334604,0.0897100242045665,r
59977.30651296135,1.6693607625310154,0.19139111634410044,z
59980.32408083299,1.6362010609126478,0.3383430741192924,r
59980.32684468052,3.0400477254291394,0.2382024591448967,z
59986.30435900956,5.019520490647114,0.3453819945395672,g
59986.30526873927,1.7233667757575966,0.2367196214614407,r
...,...,...,...
60043.15987405378,7.2987364275720195,0.2287710005225444,g


In [18]:
transient_list[9].data

,Unnamed: 0,Unnamed: 0.1,col0,XWIN_IMAGE,YWIN_IMAGE,SNR_WIN,MAGERR_APER,XWIN_WORLD,YWIN_WORLD,DetOnWhich,...,LM5_SCI,ZP_DIFF,EZP_DIFF,OAPER_DIFF,AlertID,ROUND,ObjectID,unkRa,unkDec,SEP2UNK
0,0,0,1354,353.350305,136.144903,19.282452,0.061024,0.955022,-0.645460,DIFF,...,23.637630,29.521395,0.008142,7.20,230726080420S100003492053m0038436551,47,T202307260003492m003843,NaN,NaN,NaN
1,1,1,1267,350.212457,130.763562,26.661009,0.046615,0.955036,-0.645485,DIFF,...,23.621429,31.251597,0.006454,6.23,230726080539S100003492087m0038437467,47,T202307260003492m003843,NaN,NaN,NaN


In [20]:
transient_list[9].data['SCINAME']

0    c4d_230726_080420_ooi_g_v1.fits.fz
1    c4d_230726_080539_ooi_r_v1.fits.fz
Name: SCINAME, dtype: object

In [21]:
transient_list[8].data['SCINAME']

0    c4d_230202_023207_ooi_g_desirt.fits.fz
1    c4d_230202_023504_ooi_z_desirt.fits.fz
2    c4d_230208_024036_ooi_g_desirt.fits.fz
3    c4d_230208_024157_ooi_r_desirt.fits.fz
4    c4d_230208_024337_ooi_z_desirt.fits.fz
5    c4d_230211_015135_ooi_g_desirt.fits.fz
6    c4d_230211_015255_ooi_r_desirt.fits.fz
7    c4d_230211_015434_ooi_z_desirt.fits.fz
8    c4d_230714_102321_ooi_g_desirt.fits.fz
9    c4d_230714_102440_ooi_r_desirt.fits.fz
Name: SCINAME, dtype: object

In [24]:
transient_list[8].data['SCINAME'][0][-16]

'g'

In [28]:
sciname = 'c4d_230726_080420_ooi_g'
sciname[22]

'g'

In [40]:
# including missing zspec

for lc,tran in enumerate(transient_list):
    tran.light_curve = create_lc_data3(convert_data(tran.data), tran.meta)
    print('lc',lc)
    
lc_list = [i.light_curve for i in transient_list if i.light_curve]


lc 0
lc 1
lc 2
lc 3
lc 4
lc 5
lc 6
lc 7
lc 8
lc 9
lc 10
lc 11
lc 12
lc 13
lc 14
lc 15
lc 16
lc 17
lc 18
lc 19
lc 20
lc 21
lc 22
lc 23
lc 24
lc 25
lc 26
lc 27
lc 28
lc 29
lc 30
lc 31
lc 32
lc 33
lc 34
lc 35
lc 36
lc 37
lc 38
lc 39
lc 40
lc 41
lc 42
lc 43
lc 44
lc 45
lc 46
lc 47
lc 48
lc 49
lc 50
lc 51
lc 52
lc 53
lc 54
lc 55
lc 56
lc 57
lc 58
lc 59
lc 60
lc 61
lc 62
lc 63
lc 64
lc 65
lc 66
lc 67
lc 68
lc 69
lc 70
lc 71
lc 72
lc 73
lc 74
lc 75
lc 76
lc 77
lc 78
lc 79
lc 80
lc 81
lc 82
lc 83
lc 84
lc 85
lc 86
lc 87
lc 88
lc 89
lc 90
lc 91
lc 92
lc 93
lc 94
lc 95
lc 96
lc 97
lc 98
lc 99
lc 100
lc 101
lc 102
lc 103
lc 104
lc 105
lc 106
lc 107
lc 108
lc 109
lc 110
lc 111
lc 112
lc 113
lc 114
lc 115
lc 116
lc 117
lc 118
lc 119
lc 120
lc 121
lc 122
lc 123
lc 124
lc 125
lc 126
lc 127
lc 128
lc 129
lc 130
lc 131
lc 132
lc 133
lc 134
lc 135
lc 136
lc 137
lc 138
lc 139
lc 140
lc 141
lc 142
lc 143
lc 144
lc 145
lc 146
lc 147
lc 148
lc 149
lc 150
lc 151
lc 152
lc 153
lc 154
lc 155
lc 156
lc 157
lc 1

In [91]:
dataset = lcdata.from_light_curves([i for i in lc_list_Aper])
dataset.write_hdf5('./DCDE3_1027_transients_USING_MAG_APERc_photoz_if_NoZSPEC.h5')

In [121]:
len(lc_list_OG[0])*3

117

In [90]:
len(lc_list_Aper)

1027

In [37]:
lc_list[0].meta

AttributeError: 'NoneType' object has no attribute 'meta'

In [33]:
dataset_all_wrs_ps1.light_curves[i]['band'] = dataset_all_wrs_ps1.light_curves[i]['band'].astype(np.dtype('U6'))
len(lc_list), len(transient_list)
for i in range(len(lc_list)):
lc_list.light_curves[i]['band']=.light_curves[i]['band'].astype(np.dtype('U6'))

time,flux,fluxerr,band,zp,magsys
float64,float64,float64,str5,float64,str2
60159.232011487846,1.8665858688156733e-09,1.8752609385387626e-11,lsstr,22.5,ab
60159.23293285294,1.8665858688156733e-09,1.8752609385387626e-11,lsstz,22.5,ab
60178.22611002953,8.944267924960743e-11,1.6491445200954996e-11,lsstg,22.5,ab
60178.2270241849,8.944271063476963e-11,1.649145061183891e-11,lsstz,22.5,ab
60184.20657100734,8.944267924960743e-11,1.6491445200954996e-11,lsstr,22.5,ab
60184.20772476066,8.944267924960743e-11,1.6491445200954996e-11,lsstz,22.5,ab
60187.16984580023,5.9079328014369425e-08,5.2081460479142785e-11,lsstr,22.5,ab
60187.170976911366,1.8665858688156733e-09,1.8752609385387626e-11,lsstz,22.5,ab
60202.18399681967,8.944271063476963e-11,1.649145061183891e-11,lsstz,22.5,ab


In [57]:
# try using parsnip.predict_redshift for zspec
"""def predict_redshift(self, light_curve, min_redshift=0., max_redshift=None,
                         sampling=0.01):
        Predict the redshift of a light curve.

        This evaluates the MAP estimate of the redshift.

        Parameters
        ----------
        light_curve : `~astropy.table.Table`
            Light curve

        Returns
        -------
        float
            MAP estimate of the redshift
        
        redshifts, redshift_distribution = self.predict_redshift_distribution(
            light_curve,
            min_redshift=min_redshift,
            max_redshift=max_redshift,
            sampling=sampling
        )
        return redshifts[np.argmax(redshift_distribution)]
"""
        

dataset = lcdata.from_light_curves([i for i in lc_list])
a_class = parsnip.ParsnipModel()
for lc in dataset:
    zspec = parsnip.predict_redshift(lc)
    print(zspec)
    
#parsnip.predict_redshift

TypeError: __init__() missing 2 required positional arguments: 'path' and 'bands'

In [41]:
dataset = lcdata.from_light_curves([i for i in lc_list])
dataset.write_hdf5('./DCDE4_1027_transients_photoz_if_NoZSPEC.h5')

In [53]:
model1 = parsnip.load_model('plasticc')

/Users/EmilieCote/opt/anaconda3/lib/python3.9/site-packages/parsnip/parsnip.py:297: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  self.band_interpolate_weights = torch.FloatTensor(band_weights).to(self.device)


In [ ]:
modelps1 = parsnip.load_model('ps1')

In [72]:
dataset = lcdata.read_hdf5('./DCDE5_446_transients_photoz_if_NoZSPEC.h5')

In [78]:
dataset.light_curves[40]

time,flux,fluxerr,band
float64,float32,float32,bytes1
60154.149972682535,1.5719792,0.75075066,r
60154.16545598862,4.756615,0.46048743,z
60169.23587234572,44.229294,0.026925778,g
60169.25086450697,247.41528,0.012720212,z
60175.08126404475,247.41528,0.012720212,z
60178.049757564106,274.88788,0.012884542,r
60178.06513957001,247.41528,0.012720212,z
60184.02623476856,134.41335,0.024671417,r
60184.027154308205,365.58878,0.00973005,z


In [79]:
dataset.meta[40]

object_id,ra,dec,type,redshift,ra_host,dec_host,hostgal_photoz,hostgal_photoz_err,z_spec exists
str23,float64,float64,str7,float64,float64,float64,float32,float32,int64
A202307292223500m044230,335.95858554,-4.70859539,Unknown,0.32296040654182434,335.95840506,-4.70871321,0.3229604,0.0621219,0


In [ ]:
predictions = model1.predict(dataset)

In [1]:
predictions

NameError: name 'predictions' is not defined

In [6]:
dataset[:2]

In [7]:
dataset2 = lcdata.read_hdf5('dataset_all.h5')

In [8]:
len(dataset2)

557

In [9]:
predictions = model1.predict(dataset2[:6])

KeyboardInterrupt: 